In [1]:
import pandas as pd 
import numpy as np
import os

In [2]:
image_labels = pd.read_csv("assets/data/classifications.csv")
classes = pd.read_csv("assets/data/classes.csv")

In [7]:
### Finding the id for all the clothes images
clothes = classes.loc[classes["Tortoise"] == "Clothing"]
clothing_id = clothes["/m/011k07"].item()
clothing_id

'/m/09j2d'

In [8]:
clothes = classes.loc[classes["Tortoise"] == "Clothing"]


In [9]:
### Opening the labels file 
import json
js_file = open("assets/data/labels_hierachy.json")
js_object = json.loads(js_file.read())



In [10]:

path = "/home/user/fiftyone/open-images-v6/train/data"
def img_names_to_list(path_to_files):
    filenames = [os.path.splitext(img)[0] for img in os.listdir(path_to_files)]
    return filenames
        
names = img_names_to_list(path)

In [15]:
### Importing test images
test_images = pd.DataFrame(names, columns=["ImageID"])

In [16]:
### Change the dtype of ImageID from int64 to string
image_labels["ImageID"] = image_labels["ImageID"].apply(lambda x : str(x))

In [17]:
def get_label_name(x):
    label_name = image_labels.loc[image_labels["ImageID"] == x, 'LabelName']
    print(label_name)
    


In [18]:
### Retrieve the dict containing the clothing sublabels
def find_clothes_dict():
    for i in js_object["Subcategory"]:
        if type(i) == dict:
            if i["LabelName"] == clothing_id:
                clothes_dict = i["Subcategory"]
                # print(clothes_dict, "from first level")
                return clothes_dict
        elif type(i) == list:
            for item in i:
                if item["LabelName"] == clothing_id:
                    
                    return item['LabelName']




In [19]:
len(find_clothes_dict())

19

In [20]:
### Loop over the clothign dict and collect the labels
def create_ids_list():
    ids_list = []
    for i in find_clothes_dict():
        if "Subcategory" in i:
            for d in i["Subcategory"]:
                if "Subcategory" in d:
                    for l in d["Subcategory"]:
                        if "Subcategory" in l:
                            for m in l["Subcategory"]:
                                if "Subcategory" in m:
                                    for elem in m["Subcategory"]:
                                        ids_list.extend(elem.values())
                                else:
                                    ids_list.extend(m.values())
                        else : 
                            ids_list.extend(l.values())
                else : 
                    ids_list.extend(d.values())
        else : 
            ids_list.extend(i.values())
    return ids_list

find_clothes_dict()

[{'LabelName': '/m/01bfm9'},
 {'LabelName': '/m/01d40f'},
 {'LabelName': '/m/01gkx_'},
 {'LabelName': '/m/01gmv2'},
 {'LabelName': '/m/01krhy'},
 {'LabelName': '/m/01n4qj'},
 {'LabelName': '/m/01xygc'},
 {'LabelName': '/m/01xyhv'},
 {'LabelName': '/m/02dl1y',
  'Subcategory': [{'LabelName': '/m/025rp__'},
   {'LabelName': '/m/02fq_6'},
   {'LabelName': '/m/02jfl0'},
   {'LabelName': '/m/02wbtzl'}]},
 {'LabelName': '/m/02h19r'},
 {'LabelName': '/m/02wv6h6', 'Subcategory': [{'LabelName': '/m/01cmb2'}]},
 {'LabelName': '/m/032b3c'},
 {'LabelName': '/m/0463sg',
  'Subcategory': [{'LabelName': '/m/0174n1',
    'Subcategory': [{'LabelName': '/m/03grzl'}]},
   {'LabelName': '/m/0176mf'},
   {'LabelName': '/m/017ftj'},
   {'LabelName': '/m/01krhy'},
   {'LabelName': '/m/01llwg'},
   {'LabelName': '/m/01nq26'},
   {'LabelName': '/m/01r546'},
   {'LabelName': '/m/01rkbr'},
   {'LabelName': '/m/02_n6y'},
   {'LabelName': '/m/02dl1y',
    'Subcategory': [{'LabelName': '/m/025rp__'},
     {'LabelNa

In [21]:
clothing_images = image_labels[image_labels["LabelName"].isin (create_ids_list())]
clothing_images.reset_index(inplace=True)

In [22]:
clothing_images

,index,ImageID,Source,LabelName,Confidence
0,6,000002b66c9c498e,verification,/m/017ftj,0
1,9,000002b66c9c498e,verification,/m/01d40f,1
2,12,000002b66c9c498e,verification,/m/01gkx_,0
3,20,000002b66c9c498e,verification,/m/01xyhv,0
4,87,000002b66c9c498e,verification,/m/0jyfg,0
...,...,...,...,...,...
365781,8996706,ffff54ccbcad1111,verification,/m/01n4qj,0
365782,8996714,ffff54ccbcad1111,verification,/m/0fly7,1
365783,8996722,ffff60d8b2d3082c,verification,/m/032b3c,1
365784,8996759,ffffd60a12c5dc33,verification,/m/01d40f,1


In [24]:
### Importting bounding boxes
chunksize = 100000
images_with_bb = pd.DataFrame()
with pd.read_csv("assets/data/oidv6-train-annotations-bbox.csv", chunksize=chunksize) as bb:
    for chunk in bb:
        images_i_want = chunk[chunk["ImageID"].isin(clothing_images["ImageID"])]
        images_with_bb = images_with_bb.append(images_i_want)



In [4]:
images_with_bb.to_csv('assets/data/bounding_boxes.csv')

NameError: name 'images_with_bb' is not defined

In [10]:
bb_images = pd.read_csv('assets/data/bounding_boxes.csv')

In [8]:
def labelize(x):
    label = classes.loc[classes["LabelName"]==x, "Object"]
    return label


In [ ]:
images_with_names = pd.merge(bb_images, classes, on="LabelName")

In [9]:
# new_chunksize = 800000
# with pd.read_csv('assets/data/bounding_boxes.csv', chunksize=new_chunksize) as bb_images:
#     for chunk in bb_images:
#         chunk["ObjectNames"] = chunk["ImageID"].apply(labelize)

ValueError: Wrong number of items passed 0, placement implies 1